In [2]:
# importation des librairies
! pip install -q langchain
! pip install -q openai
! pip install -q tiktoken
! pip install -q datasets cassio
! pip install -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is no

https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html

In [6]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
#from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
#from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from datasets import load_dataset
import cassio
import openai
import os

In [7]:
ASTRA_DB_APPLICATION_TOKEN = "" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "" # enter your Database ID


In [8]:
os.environ["OPENAI_API_KEY"] = "sk----"
openai.api_key =os.getenv("OPENAI_API_KEY")

In [9]:
from PyPDF2 import PdfReader

In [11]:
pdfreader = PdfReader("/content/Guide des aides rénovation 2024.pdf")

In [12]:
# Read the pdf file
raw_texte=''
for i, page in enumerate(pdfreader.pages):
  content= page.extract_text()
  if content:
    raw_texte+=content

In [13]:
raw_texte

"  Les aides \n  financières \n  en 2024 \nÉdition janvier 2024Le service public pour mieux\nrénover mon habitat\n   Des aides différentes en fonction  \nde vos revenus\n  Un accompagnement personnalisé\n  Les aides à la rénovation énergétique\n 9 MaPrimeRénov’\n 17 MaPrimeRénov’ Parcours accompagné\n 20 MaPrimeRénov’ Copropriété\n 22 Quelles règles en cas de cumul d’aides ?\n 25 Le dispositif Loc’Avantages \n 26  Les aides des fournisseurs d’énergie  \n(dispositif des certificats d’économies d’énergie)\n 31 La TVA à taux réduit\n 33 Les aides des collectivités locales\n 33 Le chèque énergie\n 34 L’exonération de taxe foncière\n 35 L’aide de votre caisse de retraite\n 35 Le dispositif Denormandie\n 37 Le déficit foncier\n 38 L’éco-prêt à taux zéro\n 42 Le prêt avance rénovation\n 42 D’autres prêts à connaître\n 43 Les critères techniques d’éligibilité des travaux\n   MaPrimeAdapt’, l’aide pour l’adaptation à la \nperte d’autonomie\n   Les aides pour la lutte contre l’habitat indigne4\n

In [17]:
# intiatialisation de la connection à Cassandra
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(139641519142320) 78b6059c-858f-43fd-822d-af5479dfdca8-us-east1.db.astra.datastax.com:29042:da4b7bf4-d0da-4d63-9400-9ed0a495c23b> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [14]:
# initiation du modele et de l'embedding
llm= ChatOpenAI(model="gpt-4",temperature=0.2,max_tokens=200)
embedding = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [18]:
astra_vector_store = Cassandra(embedding=embedding,
                            table_name="qa_mini_demo",
                            session=None,
                            keyspace=None,)

In [20]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100,length_function=len,)
texts= text_spliter.split_text(raw_texte)

In [21]:
print(f" le nombre de chunks obtenus apres division du texte total est de :{len(texts)}")
print("\n")
print("voila les 3 premiers chunks:","\n")
texts[:3]

 le nombre de chunks obtenus apres division du texte total est de :176


voila les 3 premiers chunks: 



['Les aides \n  financières \n  en 2024 \nÉdition janvier 2024Le service public pour mieux\nrénover mon habitat\n   Des aides différentes en fonction  \nde vos revenus\n  Un accompagnement personnalisé\n  Les aides à la rénovation énergétique\n 9 MaPrimeRénov’\n 17 MaPrimeRénov’ Parcours accompagné\n 20 MaPrimeRénov’ Copropriété\n 22 Quelles règles en cas de cumul d’aides ?\n 25 Le dispositif Loc’Avantages \n 26  Les aides des fournisseurs d’énergie  \n(dispositif des certificats d’économies d’énergie)\n 31 La TVA à taux réduit\n 33 Les aides des collectivités locales\n 33 Le chèque énergie\n 34 L’exonération de taxe foncière\n 35 L’aide de votre caisse de retraite\n 35 Le dispositif Denormandie\n 37 Le déficit foncier\n 38 L’éco-prêt à taux zéro\n 42 Le prêt avance rénovation\n 42 D’autres prêts à connaître',
 '38 L’éco-prêt à taux zéro\n 42 Le prêt avance rénovation\n 42 D’autres prêts à connaître\n 43 Les critères techniques d’éligibilité des travaux\n   MaPrimeAdapt’, l’aide pour l

In [22]:
len(texts)

176

In [23]:
NUM_HEADLINES = len(texts)


print("\nGenerating embeddings and storing headlines in AstraDB ... ", end="")
astra_vector_store.add_texts(texts[:NUM_HEADLINES])


astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)


Generating embeddings and storing headlines in AstraDB ... 

In [24]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:100]))


Enter your question (or type 'quit' to exit): quel montant avoir si j'installe une pompe a chaleur air/eau

QUESTION: "quel montant avoir si j'installe une pompe a chaleur air/eau"


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprec

ANSWER: "Le montant de l'aide que vous pouvez recevoir pour l'installation d'une pompe à chaleur air/eau dépend de vos ressources. Si vous êtes dans la catégorie des ménages aux ressources très modestes, vous pouvez recevoir 5 000 €. Si vous êtes dans la catégorie des ménages aux ressources modestes, vous pouvez recevoir 4 000 €. Si vous êtes dans la catégorie des ménages aux ressources intermédiaires, vous pouvez recevoir 3 000 €. Les ménages aux ressources supérieures ne sont pas éligibles pour cette aide."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9391] "Une pompe à chaleur 
air/eau4 000 € 2 500 €
Une pompe à chaleur 
eau/eau ou sol/eau5 000 €
Un systèm ..."
    [0.9388] "Chauffe-eau thermodynamique 1 200 € 800 € 400 € non éligible
Pompe à chaleur air/eau  
(dont PAC hyb ..."
    [0.9388] "ET MATÉRIAUX ÉLIGIBLESAIDE POUR LES MÉNAGES
AUX 
RESSOURCES 
TRÈS 
MODESTESAUX 
RESSOURCES 
MODESTES ..."
    [0.9388] "ET MATÉRIAUX ÉLIGIBLESAIDE POUR LES MÉNAGES
AUX 
RESSOURCES 
TRÈS 
MODESTESAUX 
RESSOURCES 
MODESTES ..."

What's your next question (or type 'quit' to exit): quel condtion doivent respceter pour beneficer des aide si je fais un rénovation globale de ma maison

QUESTION: "quel condtion doivent respceter pour beneficer des aide si je fais un rénovation globale de ma maison"


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprec

...

ANSWER: "Oui, vous pouvez bénéficier d'aides pour l'installation d'une chaudière à gaz, en particulier si vous remplacez une chaudière au charbon, au fioul ou au gaz autre qu'à condensation. Ces aides sont bonifiées pour les ménages aux revenus modestes et très modestes. Les entreprises qui vendent de l'énergie (électricité, gaz ou GPL, chaleur, froid, fioul domestique, carburants pour véhicules) peuvent vous proposer des aides pour rénover votre logement. C'est une obligation encadrée par l'État."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9293] "Une fois cette offre acceptée, vous pouvez 
signer le devis des travaux. Toutefois, si vous 
avez dé ..."
    [0.9292] "-standardisees-deconomies-denergie
Des primes « coup de pouce » pour certains travaux
Tous les ménag ..."
    [0.9286] "de combustion incompatible avec des chau -
dières individuelles au gaz à condensation ;- l’achat et  ..."
    [0.9252] "programmée de l’Anah. Si ce n’est pas le cas, 
les propriétaires peuvent percevoir une prime 
pouvan ..."

What's your next question (or type 'quit' to exit): c'est faux une chaudiere gaz ne permet pas d'avoir des aides

QUESTION: "c'est faux une chaudiere gaz ne permet pas d'avoir des aides"


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprec

ANSWER: "Selon le contexte fourni, le remplacement d'une chaudière au charbon, au fioul ou au gaz (autres qu'à condensation) par un équipement utilisant des énergies renouvelables peut bénéficier d'aides financières dans le cadre du dispositif des certificats d'économies d'énergie. Cependant, si la chaudière à gaz est déjà une chaudière à condensation, elle ne serait pas éligible pour ces aides."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9205] "de combustion incompatible avec des chau -
dières individuelles au gaz à condensation ;- l’achat et  ..."
    [0.9200] "-standardisees-deconomies-denergie
Des primes « coup de pouce » pour certains travaux
Tous les ménag ..."
    [0.9159] "programmée de l’Anah. Si ce n’est pas le cas, 
les propriétaires peuvent percevoir une prime 
pouvan ..."
    [0.9153] "Une fois cette offre acceptée, vous pouvez 
signer le devis des travaux. Toutefois, si vous 
avez dé ..."

What's your next question (or type 'quit' to exit): quit
